In [ ]:
# pip install -U langchain-community
# pip install pypdf

Fonte: [Unleashing Conversational Power: A Guide to Building Dynamic Chat Applications with LangChain, Qdrant, and Ollama (or OpenAI’s GPT-3.5 Turbo)](https://medium.com/@ingridwickstevens/langchain-chat-with-your-data-qdrant-ollama-openai-913020ec504b)

In [7]:
from langchain.document_loaders import PyPDFLoader
# loader = PyPDFLoader('../data/tokio_outubro_2024.pdf')
loader = PyPDFLoader('../data/teste.pdf')
pages = loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# define the text splitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""]
)

# Create our splits from the PDF
docs = r_splitter.split_documents(pages)

In [9]:
# MODEL_NAME = 'mistral'
MODEL_NAME = 'llama2'

In [10]:
%time

from langchain.vectorstores import Qdrant

# set up Ollama Embeddings: https://python.langchain.com/docs/integrations/text_embedding/ollama
from langchain.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model=MODEL_NAME) 

# set up the qdrant database
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

CPU times: user 4 μs, sys: 1 μs, total: 5 μs
Wall time: 26.9 μs


/tmp/ipykernel_29371/2687660139.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL_NAME)


In [11]:
##### Ollama
from langchain_community.chat_models import ChatOllama

# model name can be any model you have installed with Ollama
# complete list of models available @ Ollama: https://ollama.ai/library
llm = ChatOllama(model_name=MODEL_NAME, temperature=0)

/tmp/ipykernel_29371/4263304338.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model_name=MODEL_NAME, temperature=0)


In [12]:
question = "What are the top frameworks developers use to build LLM apps?"

In [13]:
from langchain.chains.retrieval_qa.base import RetrievalQA
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=qdrant.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

/tmp/ipykernel_29371/838842698.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain_mr({"query": question})


"I don't know the answer to your question based on the provided extract from the document. The document appears to be focused on weather-related topics, and there is no relevant text that directly answers your question about the top frameworks developers use to build LLM apps."